In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# Data Point

In [2]:
from recfldgrn.datapoint import PatientDP, RANGE_SIZE

PID = 'P4'
rec_folder = 'data/ProcData/RecFolder/'
Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
Pat

In [3]:
####################
RecName = 'A1C'
####################

df = Pat.get_df_rec(RecName)
df

,PID,ECID,RID,V,DT
0,P4,EC125,6360,NaN,NaT
1,P4,EC24,6361,NaN,NaT
2,P4,EC36,5382,2.313767,2023-04-13
3,P4,EC80,6362,NaN,NaT
4,P4,EC163,6363,NaN,NaT
5,P4,EC106,6364,NaN,NaT
6,P4,EC85,6365,NaN,NaT
7,P4,EC88,6366,NaN,NaT
8,P4,EC179,6367,NaN,NaT
9,P4,EC48,6368,NaN,NaT


# Task Selection

In [4]:
TaskName = 'Task2YearXXX'

ELIG_PAT_SET = [f'P{i}' for i in range(0, 8)]
ELIG_PAT_SET

['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']

In [5]:
import os
import pandas as pd

Tensor_folder = 'data/ProcData/TensorFolder'
TaskTensor_folder = os.path.join(Tensor_folder, TaskName)

if not os.path.exists(TaskTensor_folder): os.makedirs(TaskTensor_folder)

print(TaskTensor_folder)

data/ProcData/TensorFolder/Task2YearXXX


In [6]:
######################### Change this
RecName = 'A1C'
######################### 


df = pd.DataFrame()
for PID in ELIG_PAT_SET:
    Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
    dfx = Pat.get_df_rec(RecName)
    
    ###################################
    # TODO: make some operation to select good dfx
    dfx = dfx 
    ###################################
    
    df = pd.concat([df, dfx])
    
print(df.shape)
df

(225, 5)


,PID,ECID,RID,V,DT
0,P0,EC18,5801,9.260195,2023-04-08
1,P0,EC140,6205,NaN,NaT
2,P0,EC23,6206,NaN,NaT
3,P0,EC141,6207,NaN,NaT
4,P0,EC142,6208,NaN,NaT
...,...,...,...,...,...
3,P6,EC15,3546,16.714732,2023-04-09
4,P6,EC93,6401,NaN,NaT
5,P6,EC195,5056,46.861784,2023-03-25
6,P6,EC173,6402,NaN,NaT


# Load Meta Info

In [7]:
import os
import pandas as pd

fldgrn_folder = 'data/ProcData/FldGrnInfo'

fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')

df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)
df_FieldGrainInfo

,RecName,prefix_ids,focal_ids,field_list,field,grain,field2grain,recfield2grain,Vocab
0,A1C,"[PID, ECID]",[RID],"[V, DT]",V,A1CNumeDftGrn,V-A1CNumeDftGrn,A1C@V-A1CNumeDftGrn,"{'v2idx': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
1,A1C,"[PID, ECID]",[RID],"[V, DT]",DT,DTDftGrn,DT-DTDftGrn,A1C@DT-DTDftGrn,"{'idx2v': {0: '_padding', 1: '_missing', 2: 'Y..."


# RecFldGrn `A1C@V-A1CNumeDftGrn`

In [8]:
from recfldgrn.grains import Field2Grain_2_get_GrnStrList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnIdxList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnWgtList_Fn
from recfldgrn.utils import generate_grain_vocab_info
from recfldgrn.utils import get_compressed_df
from recfldgrn.datapoint import write_df_to_folders

In [9]:
from recfldgrn.utils import get_highorder_input_idx

In [10]:
TaskTensor_folder

'data/ProcData/TensorFolder/Task2YearXXX'

In [11]:
######################### Change this
recfield2grain = 'A1C@V-A1CNumeDftGrn'
#########################


d = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfield2grain].iloc[0]
v2idx = d['Vocab']['v2idx']
prefix_ids = d['prefix_ids']
focal_ids = d['focal_ids']

df_p = get_highorder_input_idx(df, recfield2grain, 
                               v2idx, prefix_ids, focal_ids,
                               Field2Grain_2_get_GrnIdxList_Fn, 
                               Field2Grain_2_get_GrnWgtList_Fn)



fullTaskTensor_folder = os.path.join(TaskTensor_folder, recfield2grain)
print(df.shape, fullTaskTensor_folder)
if not os.path.exists(fullTaskTensor_folder): os.makedirs(fullTaskTensor_folder)
write_df_to_folders(recfield2grain, fullTaskTensor_folder, df_p)

df_p

(225, 6) data/ProcData/TensorFolder/Task2YearXXX/A1C@V-A1CNumeDftGrn


,PID,P-EC-A1C@V-A1CNumeDftGrn_wgt
0,P0,"[[[1, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,P1,"[[[0, 0, 0, 0, 11.08853405587856, 1, 1.0, 1.0,..."
2,P2,"[[[1, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,P3,"[[[1, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,P4,"[[[1, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,P5,"[[[1, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,P6,"[[[0, 0, 0, 0.6511763706255762, 0, 0, 0.0, 0.0..."
7,P7,"[[[0, 0, 0, 0, 3.0705198239387923, 1, 1.0, 1.0..."


# RecFldGrn `A1C@DT-DTDftGrn`

In [12]:
######################### Change this
recfield2grain = 'A1C@DT-DTDftGrn'
#########################

d = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfield2grain].iloc[0]
v2idx = d['Vocab']['v2idx']
prefix_ids = d['prefix_ids']
focal_ids = d['focal_ids']

df_p = get_highorder_input_idx(df, recfield2grain, 
                               v2idx, prefix_ids, focal_ids,
                               Field2Grain_2_get_GrnIdxList_Fn, 
                               Field2Grain_2_get_GrnWgtList_Fn)


fullTaskTensor_folder = os.path.join(TaskTensor_folder, recfield2grain)
print(df.shape, fullTaskTensor_folder)
if not os.path.exists(fullTaskTensor_folder): os.makedirs(fullTaskTensor_folder)
write_df_to_folders(recfield2grain, fullTaskTensor_folder, df_p)

df_p

(225, 7) data/ProcData/TensorFolder/Task2YearXXX/A1C@DT-DTDftGrn


,PID,P-EC-A1C@DT-DTDftGrn_idx
0,P0,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
1,P1,"[[[16, 27, 38, 70, 80, 95, 138]], [[2, 23, 36,..."
2,P2,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
3,P3,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
4,P4,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
5,P5,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
6,P6,"[[[16, 27, 48, 70, 76, 97, 138]], [[2, 23, 36,..."
7,P7,"[[[16, 27, 42, 70, 77, 96, 138]]]"
